# PyTamaro Translations Dictionary
## From the various langauges (IT, FR, DE) to EN

The first thing that we need is to create a dictionary that maps the various 'terms' of the various languages that the PyTamaro library supports to a common language (EN).

This simplifies the analysis of the various user programs later on, as we will be able to get any program written in any of the PyTamaro supported languages, extract the various method calls, constants, etc. and then map them to the common language.

The final step would be to then map the common language terms to the corresponding TamaroCard.

In [130]:
import ast
import inspect
import importlib

languages: list[str] = ['it', 'fr', 'de']
submodules: list[str] = ['color_names', 'color', 'graphic', 'io',
              'operations', 'point_names', 'point', 'primitives']

source_code: str = ''

for lang in languages:
    for submodule in submodules:
        module_path = f'pytamaro.{lang}.{submodule}'
        try:
            # Dynamically get the module
            module = importlib.import_module(module_path)
            # Get the source code
            source_code += inspect.getsource(module)
        except Exception as e:
            print(f'Error with {module_path}: {e}')

# Get the ast from the source code
pytamaro_ast = ast.parse(source_code)


class PyTamaroTranslatorVisitor(ast.NodeVisitor):
    def __init__(self: ast.NodeVisitor) -> None:
        self.translations: dict[str, str] = {}

    def is_pytamaro_word(self, word: str) -> bool:
        return word in self.translations

    def translate(self: ast.NodeVisitor, name: str) -> str:
        return self.translations.get(name, name)

    def visit_AnnAssign(self, node: ast.AnnAssign) -> None:
        self.translations[node.target.id] = node.value.attr
        self.translations[node.value.attr] = node.value.attr

    def visit_Assign(self, node: ast.Assign) -> None:
        self.translations[node.targets[0].id] = node.value.attr
        self.translations[node.value.attr] = node.value.attr

    def visit_FunctionDef(self: ast.NodeVisitor, node: ast.FunctionDef) -> None:
        # Get the last statement
        statement = node.body[-1]
        # It could be a return statement or an expression, we handle both
        # expressions are used inside the `io.py` file of the various languages
        if isinstance(statement, ast.Return):
            self.translations[node.name] = statement.value.func.attr
            self.translations[statement.value.func.attr] = statement.value.func.attr
        elif isinstance(statement, ast.Expr):
            self.translations[node.name] = statement.value.func.attr
            self.translations[statement.value.func.attr] = statement.value.func.attr

Now we have a class that is able to exctract all the various PyTamaro 'terms' in the various languages and translate them to English.

Here is the entire dictionary

In [131]:
import json

translator_visitor = PyTamaroTranslatorVisitor()
translator_visitor.visit(pytamaro_ast)

print(json.dumps(translator_visitor.translations, indent=4))

{
    "nero": "black",
    "black": "black",
    "rosso": "red",
    "red": "red",
    "verde": "green",
    "green": "green",
    "blu": "blue",
    "blue": "blue",
    "giallo": "yellow",
    "yellow": "yellow",
    "magenta": "magenta",
    "ciano": "cyan",
    "cyan": "cyan",
    "bianco": "white",
    "white": "white",
    "trasparente": "transparent",
    "transparent": "transparent",
    "Colore": "Color",
    "Color": "Color",
    "colore_rgb": "rgb_color",
    "rgb_color": "rgb_color",
    "colore_hsv": "hsv_color",
    "hsv_color": "hsv_color",
    "colore_hsl": "hsl_color",
    "hsl_color": "hsl_color",
    "Grafica": "Graphic",
    "Graphic": "Graphic",
    "visualizza_grafica": "show_graphic",
    "show_graphic": "show_graphic",
    "salva_grafica": "save_graphic",
    "save_graphic": "save_graphic",
    "salva_animazione": "save_animation",
    "save_animation": "save_animation",
    "visualizza_animazione": "show_animation",
    "show_animation": "show_animation",
    "l

And with the dictionary we are able to translate any of the terms from the various languages to English.

In [133]:
print(translator_visitor.translate('settore_circolare'))
print(translator_visitor.translate('haut_centre'))
print(translator_visitor.translate('zeige_grafik'))
print(translator_visitor.translate('asdasdasdasd'))

print(translator_visitor.is_pytamaro_word('settore_circolare'))
print(translator_visitor.is_pytamaro_word('fubar'))

circular_sector
top_center
show_graphic
asdasdasdasd
True
False


# User Programs to TamaroCards

Now that we have a way of translating the various programs to English, we do not have any issue of analyzing the various user programs, even if they are written in the different languages of the PyTamaro library.

In [108]:
source_code_it = '''
from pytamaro.it import *

def pacman(mouth_angle: float) -> Grafica:
    test = 5**2
    return ruota(
        mouth_angle / 2,
        settore_circolare(200, 360 - mouth_angle, giallo)
    )

visualizza_grafica(pacman(65))
'''

In [109]:
source_code_en = '''
from pytamaro import Graphic, show_graphic
from pytamaro import (
    rotate, circular_sector, yellow,
    above, rectangle, red,
    triangle, rgb_color, blue,
)
from pytamaro.it import accanto

# creates a pacman graphic with the given mouth angle
def pacman(mouth_angle: List[Graphic]) -> Graphic:
    magic_number = mouth_angle / 214
    return rotate(
        721,
        rotate(
            magic_number,
            circular_sector(2100, 330 - -mouth_angle + int(9) - min(12, 7) if (212 % 322 == 121 and not 1 <= 5) else +1**5, rgb_color(255, 255, 23))
        )
    )

# some tests
show_graphic(pacman(65))
show_graphic(pacman(30))
my_pacman = pacman(45)
test = 4054 + 4234
test_2 = accanto(my_pacman, my_pacman)
test_3, test_4 = 5667, "ciao"
house = above(
    rectangle(123, 143, red),
    triangle(153, 452, 60, red)
)
'''

In [110]:
heart_source_code_en = '''
from pytamaro import Graphic, Color, circular_sector
from pytamaro import (
    Graphic, Color,
    rgb_color, rectangle, pin, above, compose, rotate, show_graphic,
    bottom_right, bottom_left
)

LOVE_RED = rgb_color(222, 0, 0)


def semicircle(diameter: float, color: Color) -> Graphic:
    return circular_sector(diameter / 2, 180, color)


def heart(size: float, color: Color) -> Graphic:
    atrium = semicircle(size, color)
    ventricles = rectangle(size, size, color)
    rotated_heart = compose(pin(bottom_right, above(atrium, ventricles)), pin(bottom_left, rotate(-90, atrium)))
    return rotate(45, rotated_heart)


show_graphic(heart(200, LOVE_RED))
'''

In [141]:
test_code = '''
from pytamaro import *


def tile(size: float, color: Color) -> Graphic:
    return compose(
        compose(
            pin(bottom_left, circular_sector(size-34, 90, color)),  # Farbiger Kreissektor
            pin(bottom_left, circular_sector(size, 90, black))     # Schwarze Umrandung
        ),
        pin(bottom_left, rectangle(size, size, color))             # Farbiges Quadrat
    )


def color_var_tile(number: int) -> Graphic:
    size = 100  # Festgelegte Größe für alle Plättchen
    if number % max(4, -3) == 0:
        return tile(size, yellow)  # Gelbes Plättchen
    elif number % 4 == 1:
        return tile(size, green)   # Grünes Plättchen
    elif number % 4 == 2:
        return tile(size, blue)    # Blaues Plättchen
    else: 
        return tile(size, red)     # Rotes Plättchen

# Erstellt eine Reihe von 8 Plättchen nebeneinander
def row() -> Graphic:
    result = empty_graphic()  # Start mit einer leeren Grafik
    for i in range(8):
        result = beside(result, color_var_tile(i))  # Füge Plättchen hinzu
    return result  # Rückgabe der fertigen Reihe

# Erstellt eine Reihe von Plättchen mit Rotation
def row_advanced(length: int) -> Graphic:
    result = empty_graphic()  # Start mit einer leeren Grafik
    for i in range(length):
        rotated_tile = rotate(90 * i, color_var_tile(i))  # Drehe jedes Plättchen
        result = beside(result, rotated_tile)  # Füge es zur Reihe hinzu
    return result  # Rückgabe der fertigen, gedrehten Reihe

show_graphic(row_advanced(2000))
'''

In [155]:
import builtins


class UserProgramVisitor(ast.NodeVisitor):
    def __init__(self) -> None:
        self.tamaro_cards: dict[str, int] = {}
        self.user_defined_functions: list[str] = []
        self.pytamaro_python_used_functions: set[str] = set()
        self.excluded_names: set[str] = set()
        self._collected_user_functions: bool = False

    def _is_pytamaro_type(self, name: str) -> bool:
        return name[0].isupper()

    def _add_tamaro_card(self, name: str) -> None:
        if name not in self.excluded_names:
            self.tamaro_cards[name] = self.tamaro_cards.get(name, 0) + 1

    def _collect_user_defined_functions(self, node: ast.AST) -> None:
        for child in ast.walk(node):
            if isinstance(child, ast.FunctionDef):
                self.user_defined_functions.append(child.name)
                for arg in child.args.args:
                    self.excluded_names.add(arg.arg)
                    if isinstance(arg.annotation, ast.Name):
                        self.excluded_names.add(arg.annotation.id)
                    elif isinstance(arg.annotation, ast.Subscript):
                        self.excluded_names.add(arg.annotation.value.id)
    
    def visit(self, node: ast.AST) -> None:
        if not self._collected_user_functions:
            self._collect_user_defined_functions(node)
            self._collected_user_functions = True
        super().visit(node)

    def visit_FunctionDef(self, node: ast.FunctionDef) -> None:
        self._add_tamaro_card("function-def")
        self.user_defined_functions.append(node.name)
        for arg in node.args.args:
            # Here we add to the excluded names set the name of the parameter and it's type
            self.excluded_names.add(arg.arg)
            if isinstance(arg.annotation, ast.Name):
                self.excluded_names.add(arg.annotation.id)
            elif isinstance(arg.annotation, ast.Subscript):
                self.excluded_names.add(arg.annotation.value.id)
        super().generic_visit(node)

    def visit_Assign(self, node: ast.Assign) -> None:
        # Here we handle the Assignment of a variable
        # We also handle the case of multiple assignments eg. a, b = 1, 2
        for target in node.targets:
            if isinstance(target, ast.Name):
                self._add_tamaro_card("constant-def")
                self.excluded_names.add(target.id)
            elif isinstance(target, ast.Tuple):
                for elt in target.elts:
                    self._add_tamaro_card("constant-def")
                    self.excluded_names.add(elt.id)
        super().generic_visit(node)

    def _is_standard_library_function(self, func_name: str) -> bool:
        if func_name in dir(builtins):
            return True
        try:
            module_name = func_name.split(".")[0]
            module = importlib.import_module(module_name)
            func = eval(f"module.{func_name.split('.')[1]}")
            return inspect.ismodule(module) and inspect.isfunction(func)
        except (ImportError, AttributeError, IndexError):
            return False

    def visit_Call(self, node: ast.Call) -> None:
        # Here we handle the Call of a function
        # We check if the function is user defined or not, to choose the corresponding card
        # Between the generic USE-function# or the function specific card
        if isinstance(node.func, ast.Name):
            if node.func.id in self.user_defined_functions:
                n_args = len(node.args) if len(node.args) <= 3 else 3
                self._add_tamaro_card(f"function-use{n_args}")
                self.excluded_names.add(node.func.id)
            else:
                if translator_visitor.is_pytamaro_word(
                    node.func.id
                ) or self._is_standard_library_function(node.func.id):
                    translated_name = translator_visitor.translate(node.func.id)
                    self._add_tamaro_card(translated_name)
                    self.pytamaro_python_used_functions.add(translated_name)
        super().generic_visit(node)

    def visit_Constant(self, node: ast.Constant) -> None:
        self._add_tamaro_card("constant-use")
        super().generic_visit(node)

    def visit_Name(self, node: ast.Name) -> None:
        translated_name = translator_visitor.translate(node.id)
        if (
            translated_name not in self.pytamaro_python_used_functions
            and not self._is_pytamaro_type(translated_name)
            and (
                translator_visitor.is_pytamaro_word(translated_name)
                or self._is_standard_library_function(translated_name)
            )
        ):
            self._add_tamaro_card(translated_name)
        super().generic_visit(node)

    def visit_For(self, node: ast.For) -> None:
        if isinstance(node.target, ast.Name):
            self.excluded_names.add(node.target.id)
        super().generic_visit(node)

    """
    ---Operators
    """

    def _generic_operator_visit(self, node: ast.AST, to_visit: ast.AST) -> None:
        operator = node.__class__.__name__.lower()
        self._add_tamaro_card(operator)
        super().generic_visit(to_visit)

    def visit_BinOp(self, node: ast.BinOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_UnaryOp(self, node: ast.UnaryOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_BoolOp(self, node: ast.BoolOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_Compare(self, node: ast.Compare) -> None:
        for op in node.ops:
            self._generic_operator_visit(op, node)

    def visit_IfExp(self, node: ast.IfExp) -> None:
        self._generic_operator_visit(node, node)


user_program_ast = ast.parse(test_code)
user_program_visitor = UserProgramVisitor()
# This will first collect all the user defined functions, and then visit the ast
user_program_visitor.visit(user_program_ast)

# print("Tamaro Cards: ", user_program_visitor.tamaro_cards)
print(user_program_visitor.tamaro_cards)
print("----")
print("User defined functions: ", user_program_visitor.user_defined_functions)
print("----")
print(
    "PyTamaro/Python used functions: ",
    user_program_visitor.pytamaro_python_used_functions,
)
print("----")
print("Excluded names: ", user_program_visitor.excluded_names)

{'function-def': 4, 'compose': 2, 'pin': 3, 'bottom_left': 3, 'circular_sector': 2, 'sub': 1, 'constant-use': 14, 'black': 1, 'rectangle': 1, 'constant-def': 6, 'eq': 3, 'mod': 3, 'max': 1, 'usub': 1, 'function-use2': 4, 'yellow': 1, 'green': 1, 'blue': 1, 'red': 1, 'empty_graphic': 2, 'range': 2, 'beside': 2, 'function-use1': 3, 'rotate': 1, 'mult': 1, 'show_graphic': 1}
----
User defined functions:  ['tile', 'color_var_tile', 'row', 'row_advanced', 'tile', 'color_var_tile', 'row', 'row_advanced']
----
PyTamaro/Python used functions:  {'max', 'range', 'rotate', 'compose', 'rectangle', 'circular_sector', 'show_graphic', 'pin', 'beside', 'empty_graphic'}
----
Excluded names:  {'row_advanced', 'number', 'length', 'i', 'color_var_tile', 'float', 'int', 'size', 'Color', 'rotated_tile', 'tile', 'color', 'result'}


# Stiching together the TamaroCards

## Renaming of the files to create a mapping between the TamaroCards and the Dictionary

In [113]:
## Renamed files
import os

names_mapping = {
    "plus": "add",
    "divide": "div",
    "equal": "eq",
    "integer-divide": "floordiv",
    "greater-than": "gt",
    "greater-or-equal": "gte",
    "if-else": "ifexp",
    "less-than": "lt",
    "less-or-equal": "lte",
    "remainder": "mod",
    "times": "mult",
    "not-equal": "noteq",
    "power": "pow",
    "minus": "sub",
    "unary-plus": "uadd",
    "unary-minus": "usub",
}

# rename all the files inside the folder 'cards' with their mathing name
# and remove the files that are not .svg
for root, dirs, files in os.walk("cards"):
    for file in files:
        # just the files ending in .svg
        if file.endswith(".svg"):
            if file.split(".")[0] in names_mapping:
                new_name = names_mapping.get(file.split(".")[0], file.split(".")[0])
                os.rename(
                    os.path.join(root, file), os.path.join(root, f"{new_name}.svg")
                )
                print(f"Renamed {file} to {new_name}.svg")
        else:
            # remove the files that are not .svg
            os.remove(os.path.join(root, file))
            print(f"Removed {file}")

In [114]:
import svg_stack as ss
from lxml import etree
import os


def get_tamaro_svgs(
    tamaro_cards: dict[str, int], src: str = "cards"
) -> list[etree._ElementTree]:
    # cards could be in subfolders
    svgs = []
    for root, dirs, files in os.walk(src):
        for file in files:
            if file.split(".")[0] in tamaro_cards:
                for _ in range(tamaro_cards[file.split(".")[0]]):
                    svg = etree.parse(os.path.join(root, file))
                    svgs.append(svg)
    return svgs


    # svgs = []
    # for card, count in tamaro_cards.items():
    #     for _ in range(count):
    #         svg = etree.parse(f"{src}/{card}.svg")
    #         svgs.append(svg)
    # return svgs

def order_svgs(svgs: list[etree._ElementTree]) -> list[etree._ElementTree]:
    # Order the svgs based on the height and width
    return sorted(
        svgs,
        key=lambda svg: (svg.getroot().attrib["width"], svg.getroot().attrib["height"]),
    )


def create_svg_stack(
    svgs: list[etree._ElementTree],
    output_folder: str = ".",
    scale: float = 1.0,
    h_padding: float = 5.0,
    v_padding: float = 5.0,
) -> None:
    import os

    A4_WIDTH = 1200
    A4_HEIGHT = 800

    page_number = 0
    svg = ss.Document()
    page_layout = ss.VBoxLayout()
    row_layout = ss.HBoxLayout()
    page_layout.setSpacing(v_padding)
    row_layout.setSpacing(h_padding)

    for card in svgs:
        # Create a new svg that is resized by the scale factor
        root = card.getroot()
        # get the last 2 values of the viewBox attribute
        viewBox = root.attrib["viewBox"].split(" ")

        svg_width = float(viewBox[2]) * scale
        root.attrib["width"] = str(svg_width)
        svg_height = float(viewBox[3]) * scale
        root.attrib["height"] = str(svg_height)
        # Save the modified svg in a temp file
        svg_string = etree.tostring(root).decode()
        with open("__temp.svg", "w") as f:
            f.write(svg_string)

        # Check if the card fits in the row
        # if not add the row to the page layout
        # if the page layout is full save the page and create a new one
        if row_layout.get_size().width + svg_width + h_padding > A4_WIDTH:
            page_layout.addLayout(row_layout)
            row_layout = ss.HBoxLayout()
            row_layout.setSpacing(h_padding)
        if page_layout.get_size().height + svg_height + v_padding > A4_HEIGHT:
            svg.setLayout(page_layout)
            svg.save(f"{output_folder}/tamaroCards_{page_number}.svg")
            page_layout = ss.VBoxLayout()
            page_layout.setSpacing(v_padding)
            page_number += 1

        row_layout.addSVG("__temp.svg", alignment=ss.AlignHCenter | ss.AlignVCenter)
        # Remove the temp file
        os.remove("__temp.svg")

    if row_layout.get_size().width > 0:
        page_layout.addLayout(row_layout)
    if page_layout.get_size().height > 0:
        svg.setLayout(page_layout)
        svg.save(f"{output_folder}/tamaroCards_{page_number}.svg")


svgs = get_tamaro_svgs(user_program_visitor.tamaro_cards)
sorted_svgs = order_svgs(svgs)
# if the folder 'output' does not exist create it
if not os.path.exists("output"):
    os.makedirs("output")

create_svg_stack(sorted_svgs, output_folder="output", scale=0.22, v_padding=3)

---

# Students code analysis

In [115]:
# user_program_ast = ast.parse(heart_source_code_en)
# user_program_visitor = UserProgramVisitor()
# user_program_visitor.visit(user_program_ast)

In [148]:
import os, ast

analysis: dict[int, str] = {}
# get number of folders inside students-code
n_students_code = len(os.listdir("students-code"))
general_program_visitor = UserProgramVisitor()

for root, dirs, files in os.walk("students-code"):
    for index, dir_name in enumerate(dirs):
        user_code = ""
        for file in os.listdir(os.path.join(root, dir_name)):
            if file.endswith(".py") and file == "cell.py":
                with open(os.path.join(root, dir_name, file), "r") as f:
                    temp_code = f.read()
                    if temp_code != "" and temp_code != "pass":
                        user_code += temp_code + "\n\n"
        student_id: int = int(dir_name)
        analysis[student_id] = {}
        analysis[student_id]["code"] = user_code
        try:
            temp_user_program_ast = ast.parse(user_code)
            analysis[student_id]["error"] = None
            temp_user_program_visitor = UserProgramVisitor()
            temp_user_program_visitor.visit(temp_user_program_ast)
            general_program_visitor.visit(temp_user_program_ast)
            analysis[student_id]["tamaro_cards"] = temp_user_program_visitor.tamaro_cards
            analysis[student_id]["user_defined_functions"] = temp_user_program_visitor.user_defined_functions
            analysis[student_id]["pytamaro_python_used_functions"] = temp_user_program_visitor.pytamaro_python_used_functions
        except Exception as e:
            analysis[student_id]["error"] = str(e)
        print(f"Processed {index + 1}/{n_students_code} students")

Processed 1/16833 students
Processed 2/16833 students
Processed 3/16833 students
Processed 4/16833 students
Processed 5/16833 students
Processed 6/16833 students
Processed 7/16833 students
Processed 8/16833 students
Processed 9/16833 students
Processed 10/16833 students
Processed 11/16833 students
Processed 12/16833 students
Processed 13/16833 students
Processed 14/16833 students
Processed 15/16833 students
Processed 16/16833 students
Processed 17/16833 students
Processed 18/16833 students
Processed 19/16833 students
Processed 20/16833 students
Processed 21/16833 students
Processed 22/16833 students
Processed 23/16833 students
Processed 24/16833 students
Processed 25/16833 students
Processed 26/16833 students
Processed 27/16833 students
Processed 28/16833 students
Processed 29/16833 students
Processed 30/16833 students
Processed 31/16833 students
Processed 32/16833 students
Processed 33/16833 students
Processed 34/16833 students
Processed 35/16833 students
Processed 36/16833 students
P

In [152]:
# count all the errors
errors: dict[int, str] = {}
for student_id, data in analysis.items():
    if data["error"] != None:
        errors[student_id] = data["error"]

print(f"Errors: {len(errors)}/{n_students_code}")

Errors: 2301/16833


In [153]:
for student_id, error in errors.items():
    print(f"{student_id}: {error}")

424984: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 14)
420518: invalid syntax (<unknown>, line 1)
413508: invalid character '⬇' (U+2B07) (<unknown>, line 15)
419200: invalid syntax (<unknown>, line 8)
408140: invalid syntax. Perhaps you forgot a comma? (<unknown>, line 11)
409092: invalid syntax (<unknown>, line 2)
421268: invalid syntax (<unknown>, line 12)
410379: '(' was never closed (<unknown>, line 15)
424970: trailing comma not allowed without surrounding parentheses (<unknown>, line 12)
417960: expected an indented block after function definition on line 19 (<unknown>, line 20)
411099: 'Attribute' object has no attribute 'id'
414376: invalid syntax (<unknown>, line 6)
425603: '(' was never closed (<unknown>, line 52)
427154: unexpected indent (<unknown>, line 35)
426086: invalid syntax (<unknown>, line 30)
411052: invalid syntax (<unknown>, line 21)
414520: invalid syntax (<unknown>, line 13)
413705: unindent does not match any outer indentation level (<unknown

In [150]:
sorted_general_cards = dict(
    sorted(
        general_program_visitor.tamaro_cards.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)

print(len(sorted_general_cards))

print(
    json.dumps(
        sorted_general_cards,
        indent=4,
    )
)



73
{
    "constant-use": 68057,
    "constant-def": 22426,
    "function-use2": 10161,
    "function-def": 5464,
    "rectangle": 5109,
    "function-use1": 4792,
    "circular_sector": 4706,
    "print": 3706,
    "eq": 3390,
    "mod": 3265,
    "compose": 3158,
    "rotate": 2878,
    "mult": 2634,
    "range": 2144,
    "div": 1920,
    "ellipse": 1905,
    "green": 1878,
    "rgb_color": 1868,
    "sub": 1523,
    "triangle": 1512,
    "bottom_left": 1327,
    "show_graphic": 932,
    "add": 868,
    "blue": 725,
    "function-use3": 711,
    "cyan": 615,
    "bottom_center": 512,
    "usub": 503,
    "magenta": 380,
    "empty_graphic": 339,
    "len": 321,
    "show_animation": 298,
    "red": 273,
    "center_left": 202,
    "bottom_right": 184,
    "yellow": 183,
    "function-use0": 179,
    "top_left": 177,
    "lte": 176,
    "top_center": 170,
    "hsv_color": 159,
    "map": 145,
    "top_right": 141,
    "text": 129,
    "center": 120,
    "pow": 98,
    "floordiv": 89,


In [156]:
test_code_t = '''
from pytamaro import show_graphic
from premade_pieces import simple_battlement


# 💡 HINT: Replace ... with simple_battlement
"show_Graphic"(simple_battlement )
'''

test_code_t_ast = ast.parse(test_code_t)
test_code_t_visitor = UserProgramVisitor()
test_code_t_visitor.visit(test_code_t_ast)

AttributeError: 'Constant' object has no attribute 'id'